<strong>Índice</strong>

1. Carregando Dataset
1. Amostragem aleatória simples
2. Amostragem sistemática
3. Amostragem por grupos
4. Amostragem estratificada
5. Amostragem de reservatório
6. Dados desbalanceados
    1. Classificação
    2. Naive Bayes
    3. Undersampling
    4. Oversampling

<strong>0. Carregando o nb_mypy</strong> 
<br>A extensão nb_mypy é responsável pela checagem de tipos nesse notebook. 
<br>É uma boa prática usar as chamadas type hints para auxiliar na documentação e no debug do código.
<br> <strong>PS:</strong> Note que ativar essa extensão irá deixar a performance do notebook consideravelmente mais lenta.

In [1]:
'''Load nb_mypy to enable type hints in this notebook'''
#%load_ext nb_mypy

'Load nb_mypy to enable type hints in this notebook'

In [1]:
'''Retrieve all inputs executed in the current kernel section'''
%history

'''Retrieve all inputs executed in the current kernel section'''
%history


<strong> 1. Carregando as Bibliotecas</strong> 
<br>Importando algumas bibliotecas que serão usadas ao longo do notebook.

In [3]:
import pandas as pd
import random
import numpy as np

'''import the pandas DataFrame type for use with nb_mypy extension'''
from pandas import DataFrame

<strong>2. Carregando o Conjunto de dados</strong>
<br>Leitura da base de dados e algumas verificações sobre os dados carregados.

In [4]:
'''load dataset'''
df_census_data = pd.read_csv('Bases - Curso Estatistica para Data Science\census.csv')

In [5]:
'''verify dataset rows and columns number'''
df_census_data.shape

(32561, 15)

In [6]:
'''checking the first rows from the dataset so we can take a glance at the data we are working with'''
df_census_data.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
'''checking for null values'''
df_census_data.isnull().sum()

age               0
workclass         0
final-weight      0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loos      0
hour-per-week     0
native-country    0
income            0
dtype: int64

In [8]:
'''how many unique values in each column?'''
df_census_data.nunique()

age                  73
workclass             9
final-weight      21648
education            16
education-num        16
marital-status        7
occupation           15
relationship          6
race                  5
sex                   2
capital-gain        119
capital-loos         92
hour-per-week        94
native-country       42
income                2
dtype: int64

<strong>3. Avançando na análise</strong>
<br>Agora que já sabemos quantos valores distintos existem em cada coluna, podemos listar quais valores são esses. Essa Informação pode ser útil para conhecer os diferentes valores que cada dimensão da base de dados pode assumir e, com essa informação em mãos, podemos pensar nos diferentes tipos de análises que podemos fazer a partir desses dados.

In [9]:
'''let's look at the unique values in each column'''
for col in df_census_data:
    print(df_census_data[col].unique())

[39 50 38 53 28 37 49 52 31 42 30 23 32 40 34 25 43 54 35 59 56 19 20 45
 22 48 21 24 57 44 41 29 18 47 46 36 79 27 67 33 76 17 55 61 70 64 71 68
 66 51 58 26 60 90 75 65 77 62 63 80 72 74 69 73 81 78 88 82 83 84 85 86
 87]
[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
[ 77516  83311 215646 ...  34066  84661 257302]
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
[13  9  7 14  5 10 12 11  4 16 15  3  6  2  1  8]
[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Pr

Também podemos plotar algumas estatísticas relacionadas à base de dados para melhor entender as métricas envolvidas.

In [10]:
'''let's take a glance on some stats'''
df_census_data.describe()

,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


<strong>4. Obtendo uma amostra dos dados<strong>

<strong>4.1 Amostragem aleatória</strong>
<br>Seleciona a amostra de forma aleatória.

In [11]:
def random_sampling(dataset: DataFrame, samples: int) ->  DataFrame:
    '''randomly selects a given number of samples from the dataset'''
    return dataset.sample(samples)

In [12]:
df_random_sample = random_sampling(df_census_data, 100)
df_random_sample.shape

(100, 15)

<strong>4.2 Amostragem Sistemática</strong>
<br>Divide-se o conjunto de dados pelo número de amostras para encontrar o valor da distância entre as amostras e, depois de selecionar aleatoriamente a primeira amostra, seleciona-se as demais amostras adicionando o valor da distância entre elas ao número da linha de forma sistemática.

In [13]:
def systematic_sampling(dataset: DataFrame, samples: int) -> DataFrame:
    ''' first, compute the distance from sample to sample;
        use random.seed(1) to always have the same rows selected;
        then, select the first row for our sample;
        then, select other samples based on the first selected row and the step;
        finally, return sample dataset'''
    step = len(dataset) // samples 
    #random.seed(1)
    start = random.randint(0,step)
    i_sample = np.arange(start, len(dataset), step = step)
    return dataset.iloc[i_sample]

In [14]:
df_systematic_sample = systematic_sampling(df_census_data, 100)
df_systematic_sample.shape

(100, 15)

<strong>4.3 Amostragem em Grupo</strong>
<br>Divida o conjunto de dados em grupos de mesmo tamanho e selecione um grupo para compor a amostra.

In [15]:
def group_sampling(dataset: DataFrame, n_groups: int) -> DataFrame:
    try:
        ''' first, compute the group size;
            then, create a group column to attribute each row to a group;
            then, randomly select one group;
            finally, return the sample dataset;
            note the use of .copy() to get rid of:
            SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame'''
        step = len(dataset) / n_groups
        groups = []
        id_group = 0
        index = 0
        for _ in dataset.iterrows():
            groups.append(id_group)
            index += 1
            if index > step:
                index = 0
                id_group += 1
        dataset = dataset.copy()
        dataset['groups'] = groups
        selected_group = random.randint(0, n_groups)
        temp_dataset = dataset[dataset['groups'] == selected_group].copy()
        return temp_dataset
    finally:
        ''' running only the section on 'try:' was appending the column 'groups' to the dataset
            this section drops the column 'groups' from the dataset'''
        dataset.drop('groups', axis=1, inplace=True)
        temp_dataset.drop('groups', axis=1, inplace=True)

In [16]:
df_group_sample = group_sampling(df_census_data, 326)
df_group_sample.shape

(100, 15)

<strong>4.4 Amostragem Estratificada </strong>
<br>Divide-se o conjunto de dados de acordo com alguma dimensão relevante e selecioná-se as amostras de acordo com a divisão realizada. O objetivo dessa estratificação é garantir que as proporções da dimensão escolhida seja a mesma tanto na população quanto na amostra.

In [17]:
'''import sklearn'''
from sklearn.model_selection import StratifiedShuffleSplit

In [18]:
def stratified_sampling(dataset: DataFrame, percentage: float) -> DataFrame:
    ''' first, we set the splt based on the desired relative sample size;
        then, we split the dataset based upon a column of our choice'''
    split = StratifiedShuffleSplit(test_size=percentage)
    for _, y in split.split(dataset, dataset['income']):
        df_y = dataset.iloc[y]
    return df_y

In [19]:
df_stratified_sample = stratified_sampling(df_census_data, (100 / len(df_census_data)))
df_stratified_sample.shape

(100, 15)

<strong>4.5 Amostragem de Reservatório </strong>
<br>Interessante para trabalhar com fluxo de dados, ou seja, com dados que estão sempre se atualizando, que estão sempre recebendo novas ocorrências. A ideia é selecionar os itens da amostra a medida que eles chegam na base.

In [20]:
def reservoir_sampling(dataset: DataFrame, samples: int) -> DataFrame:
    ''' First, create a stream to move through the dataset;
        Then, check the dataset size;
        Then, preload the reservoir with data from the stream
        Then, the while loop will change the values from the reservoir as it navigates through the dataset.
        Finally, return the sample dataset'''
    stream = []
    for i in range(len(dataset)):
        stream.append(i)
    i = 0
    size = len(dataset)
    reservoir = [0] * samples
    for i in range(samples):
        reservoir[i] = stream[i]
    while i < size:
        j = random.randrange(i + 1)
        if j < samples:
            reservoir[j] = stream[i]
        i += 1
    return dataset.iloc[reservoir]

In [21]:
df_reservoir_sample = reservoir_sampling(df_census_data, 100)
df_reservoir_sample.shape

(100, 15)

<strong>5. Comparações </strong>
<br>Após termos configurado as funções que calculam as amostras, precisamos comparar os resultados de cada método. Para isso, vamos usar uma versão reduzida do conjunto de dados, com apenas algumas métricas e dimensões.

In [22]:
'''Create a subset from our dataset, we don't need to work with all metrics and dimensions right now'''
df_census_data_subset = df_census_data[['income','race','sex','age']]

In [23]:
'''let's take a glance at the new dataset'''
df_census_data_subset.head()

,income,race,sex,age
0,<=50K,White,Male,39
1,<=50K,White,Male,50
2,<=50K,White,Male,38
3,<=50K,Black,Male,53
4,<=50K,Black,Female,28


In [24]:
'''let's check the count of unique values once again'''
df_census_data_subset.nunique()

income     2
race       5
sex        2
age       73
dtype: int64

In [25]:
'''run the sampling again'''
df_random_sample = random_sampling(df_census_data_subset, 100)
df_systematic_sample = systematic_sampling(df_census_data_subset, 100)
df_stratified_sample = stratified_sampling(df_census_data_subset, (100 / len(df_census_data_subset)))
df_group_sample = group_sampling(df_census_data_subset, 326)
df_reservoir_sample = reservoir_sampling(df_census_data_subset, 100)

In [26]:
'''check the shape of each sample'''
print(
    df_random_sample.shape,
    df_systematic_sample.shape,
    df_stratified_sample.shape,
    df_group_sample.shape,
    df_reservoir_sample.shape
)

(100, 4) (100, 4) (100, 4) (100, 4) (100, 4)


In [27]:
print(
    "Comparisons:"
    "\nDataset: %2.2f" % (df_census_data_subset['age'].mean()),
    "\nSimple Sample: %2.2f" % (df_random_sample['age'].mean()),
    "\nSystematic Sample: %2.2f" % (df_systematic_sample['age'].mean()),
    "\nStratified Sample: %2.2f" % (df_stratified_sample['age'].mean()),
    "\nGroup Sample: %2.2f" % (df_group_sample['age'].mean()),
    "\nReservoir Sample: %2.2f" % (df_reservoir_sample['age'].mean())
)

Comparisons:
Dataset: 38.58 
Simple Sample: 39.07 
Systematic Sample: 37.88 
Stratified Sample: 41.31 
Group Sample: 38.57 
Reservoir Sample: 36.99


In [28]:
'''let's compare the means from the sampling's methods with the dataset mean'''
varmean = df_census_data_subset['age'].mean()

m1 = df_random_sample['age'].mean()
m2 = df_systematic_sample['age'].mean()
m3 = df_stratified_sample['age'].mean()
m4 = df_group_sample['age'].mean()
m5 = df_reservoir_sample['age'].mean()

v1 = (varmean - df_random_sample['age'].mean()) / varmean
v2 = (varmean - df_systematic_sample['age'].mean()) / varmean
v3 = (varmean - df_stratified_sample['age'].mean()) / varmean
v4 = (varmean - df_group_sample['age'].mean()) / varmean
v5 = (varmean - df_reservoir_sample['age'].mean()) / varmean

print(
    "Mean Population versus Mean Sample:"
    "\nDataset: %2.2f" % (varmean),
    "\nSimple Sample: %2.2f e %2.2f%%" % (m1, v1*100),
    "\nSystematic Sample: %2.2f e %2.2f%%" % (m2, v2*100),
    "\nStratified Sample: %2.2f e %2.2f%%" % (m3, v3*100),
    "\nGroup Sample: %2.2f e %2.2f%%" % (m4, v4*100),
    "\nReservoir Sample: %2.2f e %2.2f%%" % (m5, v5*100)
)

Mean Population versus Mean Sample:
Dataset: 38.58 
Simple Sample: 39.07 e -1.27% 
Systematic Sample: 37.88 e 1.82% 
Stratified Sample: 41.31 e -7.07% 
Group Sample: 38.57 e 0.03% 
Reservoir Sample: 36.99 e 4.13%


<strong> 5.1 Avaliando resultados </strong>
<br>Comparar os resultados da média de idade das amostras com o valor da média do conjunto de dados pode nos dar uma ideia de qual dos métodos é mais eficiênte e mais adequado para obter uma amostra dos nossos dados, contudo não podemos parar por aqui. Podemos agora olhar os valores únicos obtidos em cada método de amostragem usado. 

In [29]:
'''let's count the unique values obtained in each sampling and add them to a dataset'''
df1 = pd.DataFrame([df_census_data_subset.nunique()]); df1['sample'] = 'original'
df2 = pd.DataFrame([df_random_sample.nunique()]); df2['sample'] = 'random'
df3 = pd.DataFrame([df_systematic_sample.nunique()]); df3['sample'] = 'systematic'
df4 = pd.DataFrame([df_stratified_sample.nunique()]); df4['sample'] = 'stratified'
df5 = pd.DataFrame([df_group_sample.nunique()]); df5['sample'] = 'group'
df6 = pd.DataFrame([df_reservoir_sample.nunique()]); df6['sample'] = 'reservoir'

In [30]:
''' we can concatanate all those datasets and reorder their columns 
    to obtain a table with the count of all unique values from all methods'''
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

cols = list(df.columns)
colunas = cols[-1:] + cols[:-1]

df_samples_nunique_comparison  = df[colunas]

O resultado desse cálculo varia de acordo com os valores aletórios gerados quando se usa cada método de amostragem, então é possível que isso não ocorra quando você rodar esse código, mas é provável que faltem algumas dimensões nessa conta. Por isso eu reduzi o número de dimensões anteriormente, para agora focar em apenas três.

Uma amostra que excluí uma raça ou uma identidade de gênero da população, pode ser considerada uma amostra válida?


In [31]:
df_samples_nunique_comparison

,sample,income,race,sex,age
0,original,2,5,2,73
1,random,2,5,2,42
2,systematic,2,5,2,42
3,stratified,2,4,2,46
4,group,2,3,2,41
5,reservoir,2,5,2,44


<strong>6. Limpando o ambiente </strong>
<br>As últimas linhas de código que implementamos deixaram algumas variáveis sobrando na memória, vamos dar uma olhada nelas e depois limpá-las?

In [32]:
''' This code will print a given number of variables, called n_var, created by the user inside this notebook.
    I found this code in stackoverflow and modified it to return the variables I've created.
    In order to do so, I have filtered all variables starting with "_", as it seems those are the sys variables.'''

import sys
def sizeof_fmt(num, suffix='B'):
    ''' by jan-glx, https://stackoverflow.com/questions/24455615, modified'''
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified by jan-glx'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

i = 0; n_var = 20
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1]):#[:10]:
    if i < n_var:
        if name.startswith('_') == False:
            print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
            i += 1

                df_census_data: 20.2 MiB
         df_census_data_subset:  6.1 MiB
          df_systematic_sample: 20.0 KiB
           df_reservoir_sample: 20.0 KiB
               df_group_sample: 20.0 KiB
          df_stratified_sample: 20.0 KiB
              df_random_sample: 20.0 KiB
                     DataFrame:  1.0 KiB
        StratifiedShuffleSplit:  1.0 KiB
                            df:  726.0 B
 df_samples_nunique_comparison:  726.0 B
                           Out:  640.0 B
                            In:  376.0 B
                           df3:  243.0 B
                           df4:  243.0 B
                           df6:  242.0 B
                           df1:  241.0 B
                           df2:  239.0 B
                           df5:  238.0 B
               random_sampling:  136.0 B


In [33]:
'''A simple del can clean some variable from the memory'''
del df, df1, df2, df3, df4, df5, df6, cols, colunas

In [34]:
''' Let's rewrite this last part of the notebook as a function, so we can recall it;
    Also, with a function, we don't have to deal with variables created to be used just one time.'''
def count_unique_values_v1():

    df1 = pd.DataFrame([df_census_data_subset.nunique()]); df1['sample'] = 'original'
    df2 = pd.DataFrame([df_random_sample.nunique()]); df2['sample'] = 'random'
    df3 = pd.DataFrame([df_systematic_sample.nunique()]); df3['sample'] = 'systematic'
    df4 = pd.DataFrame([df_stratified_sample.nunique()]); df4['sample'] = 'stratified'
    df5 = pd.DataFrame([df_group_sample.nunique()]); df5['sample'] = 'group'
    df6 = pd.DataFrame([df_reservoir_sample.nunique()]); df6['sample'] = 'reservoir'

    df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
    cols = list(df.columns)
    colunas = cols[-1:] + cols[:-1]

    return df[colunas]

In [35]:
count_unique_values_v1()

,sample,income,race,sex,age
0,original,2,5,2,73
1,random,2,5,2,42
2,systematic,2,5,2,42
3,stratified,2,4,2,46
4,group,2,3,2,41
5,reservoir,2,5,2,44


In [36]:
''' The last function already does the trick, but we can make it better:
    Let's add some parameters to this function'''
def count_unique_values_v2(dataset: DataFrame, sample1: DataFrame, sample2: DataFrame,
                        sample3: DataFrame, sample4: DataFrame, sample5: DataFrame) -> DataFrame:

    df1 = pd.DataFrame([dataset.nunique()]); df1['sample'] = 'original'
    df2 = pd.DataFrame([sample1.nunique()]); df2['sample'] = 'random'
    df3 = pd.DataFrame([sample2.nunique()]); df3['sample'] = 'systematic'
    df4 = pd.DataFrame([sample3.nunique()]); df4['sample'] = 'stratified'
    df5 = pd.DataFrame([sample4.nunique()]); df5['sample'] = 'group'
    df6 = pd.DataFrame([sample5.nunique()]); df6['sample'] = 'reservoir'

    df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
    cols = list(df.columns)
    colunas = cols[-1:] + cols[:-1]

    return df[colunas]

In [37]:
count_unique_values_v2(df_census_data_subset,df_random_sample,df_systematic_sample,
                    df_stratified_sample,df_group_sample,df_reservoir_sample)

,sample,income,race,sex,age
0,original,2,5,2,73
1,random,2,5,2,42
2,systematic,2,5,2,42
3,stratified,2,4,2,46
4,group,2,3,2,41
5,reservoir,2,5,2,44


In [38]:
import inspect

def retrieve_name(var):
    '''this function inspects the local variables and returns the name of the given variable as a string'''
    '''by scohe001, https://stackoverflow.com/questions/18425225, modified'''
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var][0]

In [39]:
def count_unique_values_v3(*argv: DataFrame) -> DataFrame:
    df = pd.DataFrame()
    dataset_mean = argv[0]['age'].mean()
    for dataset in argv:
        df_aux = pd.DataFrame([dataset.nunique()])
        df_aux['Mean'] = dataset['age'].mean()
        var = (dataset_mean - dataset['age'].mean())/dataset_mean
        df_aux['Var'] = "%2.2f%%" % (var*100),
        df_aux['Dataset'] = retrieve_name(dataset)
        df = pd.concat([df, df_aux], ignore_index=True)
    cols = list(df.columns)
    colunas = cols[-1:] + cols[:-1]
    return df[colunas]

In [40]:
count_unique_values_v3(df_census_data_subset,df_random_sample,df_systematic_sample,
                    df_stratified_sample,df_group_sample,df_reservoir_sample)

,Dataset,income,race,sex,age,Mean,Var
0,df_census_data_subset,2,5,2,73,38.581647,0.00%
1,df_random_sample,2,5,2,42,39.070000,-1.27%
2,df_systematic_sample,2,5,2,42,37.880000,1.82%
3,df_stratified_sample,2,4,2,46,41.310000,-7.07%
4,df_group_sample,2,3,2,41,38.570000,0.03%
5,df_reservoir_sample,2,5,2,44,36.990000,4.13%


In [41]:
def sampled_mean_and_standdev(dataset: DataFrame, m = 100 ):
    n = round(len(dataset)/100)
    sum_mean = 0
    sum_stand_dev = 0
    for i in range(0,m):
        df = dataset.sample(n)
        sum_mean += df.mean()
        sum_stand_dev += df.std()
    return sum_mean/m, sum_stand_dev/m

In [42]:
sampled_mean_and_standdev(df_census_data_subset['age'], 1000)

(38.53609509202453, 13.655468268964349)

In [43]:
def sample_size(dataset: DataFrame, z: float, e: float):
    '''
    80% -> 1,28
    85% -> 1,44
    90% -> 1,65
    95% -> 1,96
    99% -> 2,58
    '''
    mean, stand_dev = sampled_mean_and_standdev(dataset)
    E = mean*e
    n = (z*stand_dev/E)**2
    return print(round(n), mean, stand_dev, E, n)

In [44]:
sample_size(df_census_data_subset['age'],1.96,0.05)

191 38.630276073619626 13.611573904477877 1.9315138036809814 190.7801567789219


In [45]:
df_random_sample_v2 = random_sampling(df_census_data_subset, 200)
df_systematic_sample_v2 = systematic_sampling(df_census_data_subset, 200)
df_stratified_sample_v2 = stratified_sampling(df_census_data_subset, (200 / len(df_census_data_subset)))
df_group_sample_v2 = group_sampling(df_census_data_subset, round(len(df_census_data_subset) / 200))
df_reservoir_sample_v2 = reservoir_sampling(df_census_data_subset, 200)

In [46]:
count_unique_values_v3(df_census_data_subset,df_random_sample_v2,df_systematic_sample_v2,
                    df_stratified_sample_v2,df_group_sample_v2,df_reservoir_sample_v2)

,Dataset,income,race,sex,age,Mean,Var
0,df_census_data_subset,2,5,2,73,38.581647,0.00%
1,df_random_sample_v2,2,5,2,53,37.090000,3.87%
2,df_systematic_sample_v2,2,5,2,55,37.955224,1.62%
3,df_stratified_sample_v2,2,4,2,53,39.190000,-1.58%
4,df_group_sample_v2,2,5,2,54,37.780000,2.08%
5,df_reservoir_sample_v2,2,5,2,57,39.575000,-2.57%


In [47]:
sample_size(df_census_data_subset['age'],2.58,0.05)

332 38.55018404907973 13.62045815694944 1.9275092024539866 332.37620460292743


In [48]:
df_random_sample_v3 = random_sampling(df_census_data_subset, 350)
df_systematic_sample_v3 = systematic_sampling(df_census_data_subset, 350)
df_stratified_sample_v3 = stratified_sampling(df_census_data_subset, (350 / len(df_census_data_subset)))
df_group_sample_v3 = group_sampling(df_census_data_subset, round(len(df_census_data_subset) / 350))
df_reservoir_sample_v3 = reservoir_sampling(df_census_data_subset, 350)

In [49]:
count_unique_values_v3(df_census_data_subset,df_random_sample_v3,df_systematic_sample_v3,
                    df_stratified_sample_v3,df_group_sample_v3,df_reservoir_sample_v3)

,Dataset,income,race,sex,age,Mean,Var
0,df_census_data_subset,2,5,2,73,38.581647,0.00%
1,df_random_sample_v3,2,5,2,58,38.688571,-0.28%
2,df_systematic_sample_v3,2,5,2,57,38.282857,0.77%
3,df_stratified_sample_v3,2,5,2,57,37.500000,2.80%
4,df_group_sample_v3,2,5,2,54,38.749288,-0.43%
5,df_reservoir_sample_v3,2,5,2,57,38.054286,1.37%


In [50]:
df_census_data_subset['key'] = df_census_data_subset[['income','race','sex']].astype(str).agg('@'.join, axis=1)

C:\Users\natferna5\AppData\Local\Temp\ipykernel_14176\4183236942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_census_data_subset['key'] = df_census_data_subset[['income','race','sex']].astype(str).agg('@'.join, axis=1)


In [51]:
df_census_data_subset['key']

0           <=50K@ White@ Male
1           <=50K@ White@ Male
2           <=50K@ White@ Male
3           <=50K@ Black@ Male
4         <=50K@ Black@ Female
                 ...          
32556     <=50K@ White@ Female
32557        >50K@ White@ Male
32558     <=50K@ White@ Female
32559       <=50K@ White@ Male
32560      >50K@ White@ Female
Name: key, Length: 32561, dtype: object

In [52]:
def stratified_sampling(dataset: DataFrame, percentage: float) -> DataFrame:
    ''' first, we set the splt based on the desired relative sample size;
        then, we split the dataset based upon a column of our choice'''
    split = StratifiedShuffleSplit(test_size=percentage)
    for _, y in split.split(dataset, dataset['key']):
        df_y = dataset.iloc[y]
    return df_y

In [53]:
df_random_sample_v4 = random_sampling(df_census_data_subset, 350)
df_systematic_sample_v4 = systematic_sampling(df_census_data_subset, 350)
df_stratified_sample_v4 = stratified_sampling(df_census_data_subset, (350 / len(df_census_data_subset)))
df_group_sample_v4 = group_sampling(df_census_data_subset, round(len(df_census_data_subset) / 350))
df_reservoir_sample_v4 = reservoir_sampling(df_census_data_subset, 350)

In [54]:
count_unique_values_v3(df_census_data_subset,df_random_sample_v4,df_systematic_sample_v4,
                    df_stratified_sample_v4,df_group_sample_v4,df_reservoir_sample_v4)

,Dataset,income,race,sex,age,key,Mean,Var
0,df_census_data_subset,2,5,2,73,20,38.581647,0.00%
1,df_random_sample_v4,2,5,2,58,14,38.260000,0.83%
2,df_systematic_sample_v4,2,5,2,55,17,39.425714,-2.19%
3,df_stratified_sample_v4,2,5,2,58,15,38.960000,-0.98%
4,df_group_sample_v4,2,5,2,58,15,38.401709,0.47%
5,df_reservoir_sample_v4,2,4,2,59,14,39.065714,-1.25%


In [55]:
def count_unique_values_v4(*argv: DataFrame) -> DataFrame:
    df = pd.DataFrame()
    dataset_mean = argv[0]['age'].mean()
    for dataset in argv:
        df_aux = pd.DataFrame([dataset.nunique()])
        df_aux['mean'] = dataset['age'].mean()
        df_aux['var'] = ((dataset_mean - dataset['age'].mean())/dataset_mean)
        df_aux['dataset'] = retrieve_name(dataset)
        df = pd.concat([df, df_aux], ignore_index=True)
    cols = list(df.columns)
    colunas = cols[-1:] + cols[:-1]
    return df[colunas].set_index('dataset')

In [56]:
df = count_unique_values_v4(df_census_data_subset,df_random_sample_v4,df_systematic_sample_v4,
               df_stratified_sample_v4,df_group_sample_v4,df_reservoir_sample_v4)

print(df)

                         income  race  sex  age  key       mean       var
dataset                                                                  
df_census_data_subset         2     5    2   73   20  38.581647  0.000000
df_random_sample_v4           2     5    2   58   14  38.260000  0.008337
df_systematic_sample_v4       2     5    2   55   17  39.425714 -0.021877
df_stratified_sample_v4       2     5    2   58   15  38.960000 -0.009807
df_group_sample_v4            2     5    2   58   15  38.401709  0.004664
df_reservoir_sample_v4        2     4    2   59   14  39.065714 -0.012547


In [57]:
dfs = {}
for i in range(4):
    dfs[i] = count_unique_values_v4(df_census_data_subset,df_random_sample_v4,df_systematic_sample_v4,
                                    df_stratified_sample_v4,df_group_sample_v4,df_reservoir_sample_v4)

In [58]:
df_names = []
for i in dfs.keys():
    temp_df = dfs[i]
    df_names.append(temp_df)

df = pd.concat(df_names)

In [59]:
df.groupby(level=0).mean()

,income,race,sex,age,key,mean,var
dataset,,,,,,,
df_census_data_subset,2.0,5.0,2.0,73.0,20.0,38.581647,0.000000
df_group_sample_v4,2.0,5.0,2.0,58.0,15.0,38.401709,0.004664
df_random_sample_v4,2.0,5.0,2.0,58.0,14.0,38.260000,0.008337
df_reservoir_sample_v4,2.0,4.0,2.0,59.0,14.0,39.065714,-0.012547
df_stratified_sample_v4,2.0,5.0,2.0,58.0,15.0,38.960000,-0.009807
df_systematic_sample_v4,2.0,5.0,2.0,55.0,17.0,39.425714,-0.021877


In [60]:
def sampling_comparison(dataset: DataFrame, sample_size = 100, times = 100) -> DataFrame:
    dfs = {}
    for i in range(0,times):
        #get one sample of each type every time the loop restarts
        df_random_sample = random_sampling(dataset, sample_size)
        df_systematic_sample = systematic_sampling(dataset, sample_size)
        df_stratified_sample = stratified_sampling(dataset, (sample_size / len(dataset)))
        df_group_sample = group_sampling(dataset, round(len(dataset) / sample_size))
        df_reservoir_sample = reservoir_sampling(dataset, sample_size)
        
        dfs[i] = count_unique_values_v4(dataset,df_random_sample,df_systematic_sample,
                                   df_stratified_sample,df_group_sample,df_reservoir_sample)
    df_names = []
    for i in dfs.keys():
        temp_df = dfs[i]
        df_names.append(temp_df)
    df = pd.concat(df_names)
    #df.groupby(level=0).mean()
    return df.groupby(level=0).mean()

In [61]:
df = sampling_comparison(df_census_data_subset, 350, 100)

In [62]:
df_formated = df.style.format({
    'income': '{:,.0f}'.format,
    'race': '{:,.0f}'.format,
    'sex': '{:,.0f}'.format,
    'age': '{:,.0f}'.format,
    'key': '{:,.0f}'.format,
    'mean': '{:,.2f}'.format,
    'var': '{:,.2%}'.format,
})

df_formated


,income,race,sex,age,key,mean,var
dataset,,,,,,,
dataset,2,5,2,73,20,38.58,0.00%
df_group_sample,2,5,2,57,14,38.49,0.23%
df_random_sample,2,5,2,57,14,38.58,0.01%
df_reservoir_sample,2,5,2,57,14,38.61,-0.06%
df_stratified_sample,2,5,2,57,15,38.60,-0.05%
df_systematic_sample,2,5,2,56,15,38.49,0.23%
